In [180]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../JKBio/')
import TerraFunction as terra
from Helper import *
import numpy as np
from gsheets import Sheets
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
data_namespace="broad-genomics-delivery"
data_workspace="Cancer_Cell_Line_Factory_CCLF_PanCancer_PanelSeq"
proc_namespace="nci-mimoun-bi-org"
proc_workspace="PANCAN_TWIST"
source="CCLF"
source="CCLF"
recreateID=False
samplesetname = 'trial_twist2'
site="HT33MBCX2"
tsca_id="TSCA45"
date="201904"
TSCA_version="TSCA Rapid Cancer Detection Panel v2"
picard_aggregation_type_validation="PCR"
forcekeep=[]
gsheeturllist=["https://docs.google.com/spreadsheets/d/1LR8OFylVClxf0kmZpAdlVjrn3RBcfZKpNoDYtKdnHB8", "https://docs.google.com/spreadsheets/d/128dkFhL1A0GqTjmR7iMvBZE8j6ymO8krBL9WX-wUAn4"]

In [2]:
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

NameError: name 'Sheets' is not defined

In [ ]:
wfrom = dm.WorkspaceManager(data_namespace, data_workspace)
wto = dm.WorkspaceManager(proc_namespace, proc_workspace)

In [195]:
wto.delete_entity('pair',newpairs.index)

Pair(s) Index(['CCLF_RCRF1099-Tumor-SM-J1OZX_NA',
       'CCLF_CY1015-Tumor-SM-J1OZW_CCLF_CY1015-Tumor-SM-J1OZN',
       'CCLF_CY1015-Tumor-SM-J1OZZ_CCLF_CY1015-Tumor-SM-J1OZN',
       'CCLF_CY1006-Tumor-SM-J1OZK_NA', 'CCLF_CY1006-Tumor-SM-J1OZR_NA',
       'CCLF_RCRF1102-Tumor-SM-J1OZS_NA', 'CCLF_RCRF1102-Normal-SM-J1P12_NA',
       'CCLF_PEDS1141-Normal-SM-J1OZO_CCLF_PEDS1141-Tumor-SM-J1OZQ',
       'CCLF_SS1020-Tumor-SM-J1OZD_CCLF_SS1020-Normal-SM-J1OZA',
       'CCLF_RCRF1097-Tumor-SM-J1OZ4_NA', 'CCLF_KL1292-Tumor-SM-J1OZT_NA',
       'CCLF_PEDS1140-Tumor-SM-J1OZ5_NA', 'CCLF_PEDS1156-Tumor-SM-J1OZV_NA',
       'CCLF_RCRF1064-Tumor-SM-J1OZI_NA', 'CCLF_RCRF1070-Tumor-SM-J1OZH_NA',
       'CCLF_AB1065-Tumor-SM-J1OYT_NA', 'CCLF_PEDS1154-Tumor-SM-J1OZY_NA',
       'CCLF_PEDS1153-Tumor-SM-J1OZB_CCLF_PEDS1153-Normal-SM-J1OZ8',
       'SP014-Unknown-SM-J1OZL_NA', 'CCLF_cRCRF1085-Tumor-SM-J1OZ9_NA',
       'CCLF_cRCRF1085-Tumor-SM-J1OZC_NA', 'CCLF_cRCRF1060-Tumor-SM-J1OYS_NA',
       'CCLF_

In [196]:
wto.delete_sample(sample_info.index)

Sample(s) Index(['CCLF_cRCRF1040-Tumor-SM-J1OYQ', 'CCLF_BU1017-Normal-SM-J1OYR',
       'CCLF_cRCRF1060-Tumor-SM-J1OYS', 'CCLF_AB1065-Tumor-SM-J1OYT',
       'CCLF_KL1288-Tumor-SM-J1OYU', 'CCLF_KL1283-Tumor-SM-J1OYV',
       'CCLF_cRCRF1040-Tumor-SM-J1OYW', 'CCLF_KL1282-Tumor-SM-J1OYX',
       'CCLF_BU1013-Tumor-SM-J1OYY', 'CCLF_cRCRF1060-Tumor-SM-J1OYZ',
       'CCLF_cRCRF1060-Tumor-SM-J1OZ1', 'CCLF_cRCRF1060-Tumor-SM-J1OZ2',
       'CCLF_PEDS1143-Tumor-SM-J1OZ3', 'CCLF_RCRF1097-Tumor-SM-J1OZ4',
       'CCLF_PEDS1140-Tumor-SM-J1OZ5', 'CCLF_cRCRF1081-Tumor-SM-J1OZ7',
       'CCLF_PEDS1153-Normal-SM-J1OZ8', 'CCLF_cRCRF1085-Tumor-SM-J1OZ9',
       'CCLF_SS1020-Normal-SM-J1OZA', 'CCLF_PEDS1153-Tumor-SM-J1OZB',
       'CCLF_cRCRF1085-Tumor-SM-J1OZC', 'CCLF_SS1020-Tumor-SM-J1OZD',
       'CCLF_KL1294-Tumor-SM-J1OZE', 'CCLF_KL1235-Tumor-SM-J1OZF',
       'CCLF_AB1097-Tumor-SM-J1OZG', 'CCLF_RCRF1070-Tumor-SM-J1OZH',
       'CCLF_RCRF1064-Tumor-SM-J1OZI', 'CCLF_PEDS1064-Normal-SM-J1OZJ',
     

In [197]:

# we look at all the samples we already have
refsamples = wto.get_samples()
refids = refsamples.index

# we use this gsheet package to get all the sheets into one dataframe
metadata = pd.concat([sheets.get(url).sheets[0].to_frame() for url in gsheeturllist])

# we do some corrections just in case
samples1 = wfrom.get_samples().replace(np.nan, '', regex=True)

# renaming
metadata = metadata.dropna(0, subset=['Collaborator Sample ID'])
ttype = [replace[i.split('_')[1][-1]] for i in metadata["Collaborator Sample ID"]]
metadata['sample_id'] = [ID + '-' + ttype[i] + '-' + metadata.iloc[i]['Exported DNA SM-ID'] for i, ID in enumerate(metadata['Collaborator Participant ID'])]
samples1.index = recreateSampleID(samples1.index)

2019-08-08 03:58:52::WARNING  file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file impor

In [219]:
samples1['sample_type']

v1_Custo-Unknown-LCSET-15778     Unknown
CCLF_CY1001-Tumor-SM-IF6GG         Tumor
CCLF_KL1187-Tumor-SM-IF6GH         Tumor
CCLF_KL1212-Tumor-SM-IF6GI         Tumor
CCLF_KL1205-Tumor-SM-IF6GJ         Tumor
CCLF_PEDS1110-Tumor-SM-IF6GK       Tumor
CCLF_PEDS1106-Normal-SM-IF6GR     Normal
CCLF_AA1090-Tumor-SM-IF6GS         Tumor
CCLF_PEDS1101-Tumor-SM-IF6GT       Tumor
CCLF_KL1194-Tumor-SM-IF6GU         Tumor
CCLF_KL1189-Tumor-SM-IF6GV         Tumor
CCLF_AB1070-Tumor-SM-IF6GW         Tumor
CCLF_RCRf1060-Normal-SM-IF6H4     Normal
CCLF_CY1001-Tumor-SM-IF6H5         Tumor
CCLF_KL1202-Tumor-SM-IF6H6         Tumor
CCLF_AA1122-Tumor-SM-IF6H7         Tumor
CCLF_KL1193-Tumor-SM-IF6H8         Tumor
CCLF_AA1122-Tumor-SM-IF6H9         Tumor
CCLF_PEDS1064-Normal-SM-IF6HG     Normal
CCLF_BU1011-Normal-SM-IF6HH       Normal
CCLF_KL1189-Tumor-SM-IF6HI         Tumor
CCLF_KL1202-Tumor-SM-IF6HJ         Tumor
CCLF_KL1205-Tumor-SM-IF6HK         Tumor
CCLF_AA1090-Tumor-SM-IF6HL         Tumor
CCLF_PEDS1083-No

In [198]:
# filtering
newsamples = samples1[(~samples1.index.isin(refids)) | samples1.index.isin(forcekeep)]
newsamples['SM_ID'] = ['SM-' + i.split('-SM-')[-1] for i in newsamples.index]
tokeep = set(metadata['Exported DNA SM-ID']) & set(newsamples['SM_ID'])

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [199]:
tokeep

{'SM-J1OYQ',
 'SM-J1OYR',
 'SM-J1OYS',
 'SM-J1OYT',
 'SM-J1OYU',
 'SM-J1OYV',
 'SM-J1OYW',
 'SM-J1OYX',
 'SM-J1OYY',
 'SM-J1OYZ',
 'SM-J1OZ1',
 'SM-J1OZ2',
 'SM-J1OZ3',
 'SM-J1OZ4',
 'SM-J1OZ5',
 'SM-J1OZ7',
 'SM-J1OZ8',
 'SM-J1OZ9',
 'SM-J1OZA',
 'SM-J1OZB',
 'SM-J1OZC',
 'SM-J1OZD',
 'SM-J1OZE',
 'SM-J1OZF',
 'SM-J1OZG',
 'SM-J1OZH',
 'SM-J1OZI',
 'SM-J1OZJ',
 'SM-J1OZK',
 'SM-J1OZL',
 'SM-J1OZM',
 'SM-J1OZN',
 'SM-J1OZO',
 'SM-J1OZP',
 'SM-J1OZQ',
 'SM-J1OZR',
 'SM-J1OZS',
 'SM-J1OZT',
 'SM-J1OZU',
 'SM-J1OZV',
 'SM-J1OZW',
 'SM-J1OZX',
 'SM-J1OZY',
 'SM-J1OZZ',
 'SM-J1P11',
 'SM-J1P12'}

In [200]:

newsamples = newsamples[newsamples['SM_ID'].isin(tokeep)]
metadata = metadata[metadata['Exported DNA SM-ID'].isin(tokeep)]

newsamples = newsamples.set_index('SM_ID')
newmetadata = metadata.set_index('Exported DNA SM-ID')

In [116]:
def recreateSampleID(listLike):
      return [i.split('_')[3] + '_' + i.split('_')[4][:-1] + '-' + replace[i.split('_')[4][-1]] + '-' + i.split('_')[2] for i in listLike]

In [202]:
newsamples.shape

(46, 220)

In [205]:
newsamples

,LC-SET,PDO,aligned_reads,at_dropout,bad_cycles,bait_set,category,crai_or_bai_path,cram_or_bam_path,data_type,...,width_of_20_percent,width_of_30_percent,width_of_40_percent,width_of_50_percent,width_of_60_percent,width_of_70_percent,width_of_80_percent,width_of_90_percent,width_of_99_percent,window_size
SM_ID,,,,,,,,,,,,,,,,,,,,,
SM-J1OYQ,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,51,69,91,113,137,167,219,471,0
SM-J1OYR,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,51,69,89,111,135,165,219,485,0
SM-J1OYS,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,53,71,93,113,139,169,221,451,0
SM-J1OYT,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,25,37,51,69,89,121,163,215,333,0
SM-J1OYU,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,49,65,85,105,129,157,205,413,0
SM-J1OYV,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,31,47,65,83,101,125,151,195,389,0
SM-J1OYW,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,35,53,71,93,113,139,169,221,471,0
SM-J1OYX,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,33,49,65,85,105,127,155,203,407,0
SM-J1OYY,LCSET-15778,PDO-18546,0,0,0,BroadPanCancer2019,PAIR,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,Custom_Selection,...,29,43,59,73,91,107,135,181,409,0


In [212]:
df = pd.concat([newmetadata, newsamples], axis=1,sort=True)

In [213]:
print('creating new df')
# from this new set we create a dataframe which will get uploaded to terra
sample_info = df[['crai_or_bai_path', 'cram_or_bam_path']]
sample_info['reference_id'] = df.index
sample_info['participant'] = df['Collaborator Participant ID']
sample_info['aggregation_product_name_validation'] = [TSCA_version] * sample_info.shape[0]
sample_info['bsp_sample_id_validation'] = sample_info['reference_id']
sample_info['stock_sample_id_validation'] = df['Stock DNA SM-ID']
sample_info['sample_type'] = df['Sample Type']
sample_info['picard_aggregation_type_validation'] = [picard_aggregation_type_validation] * sample_info.shape[0]
sample_info['tumor_subtype'] = df['Tumor Type']
sample_info['source_subtype_validation'] = df['Original Material Type']
sample_info['processed_subtype_validation'] = df['Material Type']
sample_info['primary_disease'] = df['Primary Disease']
sample_info['media'] = df['Media on Tube']
sample_info['Collection'] = df['Collection']
sample_info['tissue_site'] = df['Tissue Site']
sample_info['source'] = [source] * sample_info.shape[0]
sample_info['sample_id'] = df['sample_id']

sample_info = sample_info.set_index('sample_id')

creating new df


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [214]:
normals = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
tumors = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]
prevtumors = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Tumor"]
prevnormals = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Normal"]

print("creating new pairs")

creating new pairs


In [185]:
newpairs = {'pair_id': [], 'case_sample': [], 'control_sample': [], 'participant':[]}

paired = set(tumors) & set(normals)
for val in set(tumors):
    for tumor_id in sample_info[sample_info['participant'] == val][sample_info['sample_type'] == 'Tumor'].index.tolist():
        normal_id = sample_info[(sample_info['participant'] == val) & (sample_info['sample_type'] == 'Normal')].index.tolist()[0] if val in paired else 'NA'
        newpairs['pair_id'].append(tumor_id + "_" + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [215]:
sample_info[sample_info['sample_type']=='Tumor']

,crai_or_bai_path,cram_or_bam_path,reference_id,participant,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,tumor_subtype,source_subtype_validation,processed_subtype_validation,primary_disease,media,Collection,tissue_site,source
sample_id,,,,,,,,,,,,,,,,,
CCLF_cRCRF1040-Tumor-SM-J1OYQ,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYQ,CCLF_cRCRF1040,TSCA Rapid Cancer Detection Panel v2,SM-J1OYQ,SM-IKHT2,Tumor,PCR,Metastatic,Tissue:Fresh Tissue,DNA:DNA Somatic,Non-small cell lung cancer,NaN,Cancer Cell Line Factory (CCLF) / David Sandak...,Abdominal Cavity,CCLF
CCLF_cRCRF1060-Tumor-SM-J1OYS,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYS,CCLF_cRCRF1060,TSCA Rapid Cancer Detection Panel v2,SM-J1OYS,SM-IKHT5,Tumor,PCR,Metastatic,Tissue:Fresh Tissue,DNA:DNA Somatic,Ovarian carcinoma,SMGM_M87_BCXJ,Cancer Cell Line Factory (CCLF) / David Sandak...,Brain,CCLF
CCLF_AB1065-Tumor-SM-J1OYT,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYT,CCLF_AB1065,TSCA Rapid Cancer Detection Panel v2,SM-J1OYT,SM-IKHV7,Tumor,PCR,Primary,"Cells:Cell Line, Viable",DNA:DNA Somatic,Stomach Adenocarcinoma,OPAC,Cancer Cell Line Factory (CCLF) / Adam Bass/Ku...,Stomach,CCLF
CCLF_KL1288-Tumor-SM-J1OYU,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYU,CCLF_KL1288,TSCA Rapid Cancer Detection Panel v2,SM-J1OYU,SM-IKHT7,Tumor,PCR,Metastatic,"Cells:Cell Line, Viable",DNA:DNA Somatic,Small cell lung carcinoma,NaN,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,Brain,CCLF
CCLF_KL1283-Tumor-SM-J1OYV,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYV,CCLF_KL1283,TSCA Rapid Cancer Detection Panel v2,SM-J1OYV,SM-IKHVI,Tumor,PCR,Metastatic,"Cells:Cell Line, Viable",DNA:DNA Somatic,Breast invasive carcinoma,NaN,Cancer Cell Line Factory (CCLF) / Keith Ligon ...,Pleural Cavity,CCLF
CCLF_cRCRF1040-Tumor-SM-J1OYW,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYW,CCLF_cRCRF1040,TSCA Rapid Cancer Detection Panel v2,SM-J1OYW,SM-IK3SS,Tumor,PCR,Metastatic,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,NaN,Cancer Cell Line Factory (CCLF) / David Sandak...,Lung,CCLF
CCLF_BU1013-Tumor-SM-J1OYY,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYY,CCLF_BU1013,TSCA Rapid Cancer Detection Panel v2,SM-J1OYY,SM-IK3SU,Tumor,PCR,Primary,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,SMGM_M87_BCXJ,Cancer Cell Line Factory (CCLF) / Josh Campbel...,Breast,CCLF
CCLF_cRCRF1060-Tumor-SM-J1OYZ,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OYZ,CCLF_cRCRF1060,TSCA Rapid Cancer Detection Panel v2,SM-J1OYZ,SM-IK3SQ,Tumor,PCR,Metastatic,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,RETM_AR5_BCXJ,Cancer Cell Line Factory (CCLF) / David Sandak...,Pleural Cavity,CCLF
CCLF_cRCRF1060-Tumor-SM-J1OZ1,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,gs://fc-51648132-739d-4b71-9800-03fb9f9990c6/C...,SM-J1OZ1,CCLF_cRCRF1060,TSCA Rapid Cancer Detection Panel v2,SM-J1OZ1,SM-IK3SR,Tumor,PCR,Metastatic,Tissue:Fresh Tissue,DNA:DNA Somatic,Breast invasive carcinoma,SMGM_M87_BCXJ,Cancer Cell Line Factory (CCLF) / David Sandak...,Pleural Cavity,CCLF


In [154]:

toreprocess_normals = set(tumors) & set(prevnormals)
for val in toreprocess_normals:
    for tumor_id in sample_info[sample_info['participant'] == val][sample_info['sample_type'] == 'Tumor'].index.tolist():
        normal_id = refsamples[refsamples['participant'] == val][refsamples['sample_type'] == 'Normal'].index.tolist()[0]
        newpairs['pair_id'].append(tumor_id + '_' + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

toreprocess_tumors = set(normals) & set(prevtumors)
for val in toreprocess_tumors:
    for tumor_id in refsamples[refsamples['participant'] == val][refsamples['sample_type'] == 'Tumor'].index.tolist():
        normal_id = sample_info[sample_info['participant'] == val][sample_info['sample_type'] == 'Normal'].index.tolist()[0]
        newpairs['pair_id'].append(tumor_id + '_' + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

paired = set(tumors) & set(normals)
for val in set(tumors) - (toreprocess_normals | toreprocess_tumors):
    for tumor_id in sample_info[sample_info['participant'] == val][sample_info['sample_type'] == 'Tumor'].index.tolist():
        normal_id = sample_info[(sample_info['participant'] == val) & (sample_info['sample_type'] == 'Normal')].index.tolist()[0] if val in paired else None
        newpairs['pair_id'].append(tumor_id + "_" + normal_id)
        newpairs['case_sample'].append(tumor_id)
        newpairs['control_sample'].append(normal_id)
        newpairs['participant'].append(val)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


IndexError: list index out of range

In [120]:
wto.upload_samples(sample_info)

Successfully imported 32 participants.


/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Successfully imported 46 samples.


In [106]:
wto.update_sample_set(sample_set_id=samplesetname +"_tumors", sample_ids=tumors)
#wto.update_sample_set(sample_set_id=samplesetname + "_normals", sample_ids=normals)

In [121]:
wto.update_sample_set(sample_set_id=samplesetname + "_all", sample_ids=sample_info.index.values)

Sample set "trial_twist2_all" (46 samples) successfully updated.


In [128]:
normalsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
tumorsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]

In [140]:
wto.update_sample_set(sample_set_id=samplesetname + "_tumors", sample_ids=tumorsid)

Successfully imported 1 sample sets:
  * trial_twist2_tumors (38 samples)


In [141]:
wto.update_sample_set(sample_set_id=samplesetname + "_normals", sample_ids=normalsid)

Successfully imported 1 sample sets:
  * trial_twist2_normals (8 samples)


In [144]:
normalsid.extend([k for k, val in refsamples.iterrows() if val.sample_type == "Normal"])

In [149]:
wto.update_sample_set(sample_set_id="All_normals", sample_ids=normalsid)

Successfully imported 1 sample sets:
  * All_normals (16 samples)


In [150]:
wto.update_sample_set(sample_set_id="All_samples", sample_ids = wto.get_samples().index.tolist())

Successfully imported 1 sample sets:
  * All_samples (93 samples)


In [187]:
newpairs = pd.DataFrame(newpairs).set_index('pair_id')

In [190]:
newpairs

,case_sample,control_sample,participant
pair_id,,,
CCLF_RCRF1099-Tumor-SM-J1OZX_NA,CCLF_RCRF1099-Tumor-SM-J1OZX,NA,CCLF_RCRF1099
CCLF_CY1015-Tumor-SM-J1OZW_CCLF_CY1015-Tumor-SM-J1OZN,CCLF_CY1015-Tumor-SM-J1OZW,CCLF_CY1015-Tumor-SM-J1OZN,CCLF_CY1015
CCLF_CY1015-Tumor-SM-J1OZZ_CCLF_CY1015-Tumor-SM-J1OZN,CCLF_CY1015-Tumor-SM-J1OZZ,CCLF_CY1015-Tumor-SM-J1OZN,CCLF_CY1015
CCLF_CY1006-Tumor-SM-J1OZK_NA,CCLF_CY1006-Tumor-SM-J1OZK,NA,CCLF_CY1006
CCLF_CY1006-Tumor-SM-J1OZR_NA,CCLF_CY1006-Tumor-SM-J1OZR,NA,CCLF_CY1006
CCLF_RCRF1102-Tumor-SM-J1OZS_NA,CCLF_RCRF1102-Tumor-SM-J1OZS,NA,CCLF_RCRF1102
CCLF_RCRF1102-Normal-SM-J1P12_NA,CCLF_RCRF1102-Normal-SM-J1P12,NA,CCLF_RCRF1102
CCLF_PEDS1141-Normal-SM-J1OZO_CCLF_PEDS1141-Tumor-SM-J1OZQ,CCLF_PEDS1141-Normal-SM-J1OZO,CCLF_PEDS1141-Tumor-SM-J1OZQ,CCLF_PEDS1141
CCLF_SS1020-Tumor-SM-J1OZD_CCLF_SS1020-Normal-SM-J1OZA,CCLF_SS1020-Tumor-SM-J1OZD,CCLF_SS1020-Normal-SM-J1OZA,CCLF_SS1020


In [188]:
wto.upload_entities('pair', newpairs)

Successfully imported 38 pairs.


In [189]:
wto.upload_entities('pair_set', newpairs, index=True)

Successfully imported 38 pair sets:
  * CCLF_AB1065-Tumor-SM-J1OYT_NA (1 pairs)
  * CCLF_AB1097-Tumor-SM-J1OZG_NA (1 pairs)
  * CCLF_BU1013-Tumor-SM-J1OYY_NA (1 pairs)
  * CCLF_CY1006-Tumor-SM-J1OZK_NA (1 pairs)
  * CCLF_CY1006-Tumor-SM-J1OZR_NA (1 pairs)
  * CCLF_CY1007-Tumor-SM-J1OZM_NA (1 pairs)
  * CCLF_CY1007-Tumor-SM-J1OZU_NA (1 pairs)
  * CCLF_CY1015-Tumor-SM-J1OZW_CCLF_CY1015-Tumor-SM-J1OZN (1 pairs)
  * CCLF_CY1015-Tumor-SM-J1OZZ_CCLF_CY1015-Tumor-SM-J1OZN (1 pairs)
  * CCLF_KL1235-Tumor-SM-J1OZF_NA (1 pairs)
  * CCLF_KL1283-Tumor-SM-J1OYV_NA (1 pairs)
  * CCLF_KL1288-Tumor-SM-J1OYU_NA (1 pairs)
  * CCLF_KL1292-Tumor-SM-J1OZT_NA (1 pairs)
  * CCLF_KL1294-Tumor-SM-J1OZE_NA (1 pairs)
  * CCLF_PEDS1140-Tumor-SM-J1OZ5_NA (1 pairs)
  * CCLF_PEDS1141-Normal-SM-J1OZO_CCLF_PEDS1141-Tumor-SM-J1OZQ (1 pairs)
  * CCLF_PEDS1143-Normal-SM-J1OZP_NA (1 pairs)
  * CCLF_PEDS1143-Tumor-SM-J1OZ3_NA (1 pairs)
  * CCLF_PEDS1153-Tumor-SM-J1OZB_CCLF_PEDS1153-Normal-SM-J1OZ8 (1 pairs)
  * CCLF_PEDS11

In [184]:
wto.upload_samples(pd.DataFrame({'sample_id':['NA'],'bam_filename':['NA'],'participant':['NA']}).set_index('sample_id'))

Successfully imported 1 participants.
Successfully imported 1 samples.


In [108]:
#print("uploading new samples")
wto.upload_samples(sample_info)
print("creating a sample set")
wto.delete_entity('pair',[ID + '_NA' for ID in toreprocess_tumors])
wto.update_sample_set(sample_set_id=samplesetname + "_all", sample_ids=sample_info.index.values)
wto.update_sample_set(sample_set_id=samplesetname + "_tumors", sample_ids=tumors)
wto.update_sample_set(sample_set_id=samplesetname + "_normals", sample_ids=normals)
normals.extend(prevnormals)
# Same as cum pon but better
wto.update_sample_set(sample_set_id="All_normals", sample_ids=normals)
wto.update_sample_set(sample_set_id="All_samples", sample_ids=wto.get_samples().index.tolist())
newpairs = pd.DataFrame(newpairs).set_index('pair_id')
wto.upload_entities('pair', newpairs)
wto.upload_entities('pair_set', newpairs, index=True)

creating a sample set
Pair(s) [] successfully deleted.


In [375]:
pd.DataFrame(newpairs).set_index('pair_id')

,tumor_id,normal_id
pair_id,,
CCLF_cRCRF1040-Tumor-SM-J1OYQ_NA,CCLF_cRCRF1040-Tumor-SM-J1OYQ,NA
CCLF_cRCRF1040-Tumor-SM-J1OYW_NA,CCLF_cRCRF1040-Tumor-SM-J1OYW,NA
CCLF_cRCRF1060-Tumor-SM-J1OYS_NA,CCLF_cRCRF1060-Tumor-SM-J1OYS,NA
CCLF_cRCRF1060-Tumor-SM-J1OYZ_NA,CCLF_cRCRF1060-Tumor-SM-J1OYZ,NA
CCLF_cRCRF1060-Tumor-SM-J1OZ1_NA,CCLF_cRCRF1060-Tumor-SM-J1OZ1,NA
CCLF_cRCRF1060-Tumor-SM-J1OZ2_NA,CCLF_cRCRF1060-Tumor-SM-J1OZ2,NA
CCLF_AB1065-Tumor-SM-J1OYT_NA,CCLF_AB1065-Tumor-SM-J1OYT,NA
CCLF_KL1288-Tumor-SM-J1OYU_NA,CCLF_KL1288-Tumor-SM-J1OYU,NA
CCLF_KL1283-Tumor-SM-J1OYV_NA,CCLF_KL1283-Tumor-SM-J1OYV,NA


In [277]:
metadata['External ID']

0                  CCLF_AB1065T_OPAC_P8_3D
1                 CCLF_AB1097T_ASC_Primary
2         CCLF_BU1013T_SMGM_M87_BCXJ_P4_3D
3                     CCLF_BU1017N_Primary
4                  CCLF_cRCRF1040T_Primary
5                  CCLF_cRCRF1040T_Primary
6      CCLF_cRCRF1060T_RETM_AR5_BCXJ_P3_3D
7     CCLF_cRCRF1060T_RETM_SMGM_BCXJ_P3_3D
8      CCLF_cRCRF1060T_SMGM_M87_BCXJ_P3_3D
9      CCLF_cRCRF1060T_SMGM_M87_BCXJ_P3_3D
10                 CCLF_cRCRF1081T_Primary
11               CCLF_cRCRF1085T_AR5_P6_2D
12              CCLF_cRCRF1085T_SMGM_P6_2D
13                    CCLF_CY1006T_Primary
14        CCLF_CY1006T_RETM_M87_BCXJ_P3_2D
15                    CCLF_CY1007T_Primary
16                 CCLF_CY1007T_SMGM_P5_2D
17                    CCLF_CY1015T_Primary
18                    CCLF_CY1015T_Primary
19                 CCLF_CY1015T_SMGM_P4_2D
20                    CCLF_KL1235T_Primary
21                    CCLF_KL1282T_Primary
22                    CCLF_KL1283T_Primary
23         

In [130]:
samples1 = wfrom.get_samples().replace(np.nan, '', regex=True)

In [131]:
samples1.index = [i.split('_')[3] + '_' + i.split('_')[4][:-1] + '-' + replace[i.split('_')[4][-1]] + '-' + i.split('_')[2] for i in samples1.index]

In [133]:
newsamples = samples1[~samples1.index.isin(refids)]

In [135]:
newsamples['SM_ID'] = ['SM-'+i.split('-SM-')[-1] for i in newsamples.index]

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [269]:
metadata['Exported DNA SM-ID']

0     SM-J1OYT
1     SM-J1OZG
2     SM-J1OYY
3     SM-J1OYR
4     SM-J1OYQ
5     SM-J1OYW
6     SM-J1OYZ
7     SM-J1OZ2
8     SM-J1OYS
9     SM-J1OZ1
10    SM-J1OZ7
11    SM-J1OZ9
12    SM-J1OZC
13    SM-J1OZR
14    SM-J1OZK
15    SM-J1OZU
16    SM-J1OZM
17    SM-J1OZN
18    SM-J1OZW
19    SM-J1OZZ
20    SM-J1OZF
21    SM-J1OYX
22    SM-J1OYV
23    SM-J1OYU
24    SM-J1OZT
25    SM-J1OZE
26    SM-J1OZJ
27    SM-J1OZ5
28    SM-J1OZO
29    SM-J1OZQ
30    SM-J1OZP
31    SM-J1OZ3
32    SM-J1OZ8
33    SM-J1OZB
34    SM-J1OZY
35    SM-J1P11
36    SM-J1OZV
37    SM-J1OZ6
38    SM-J1OZI
39    SM-J1OZH
40    SM-J1OZ4
41    SM-J1OZX
42    SM-J1P12
43    SM-J1OZS
44    SM-J1OZA
45    SM-J1OZD
47    SM-J1OZL
Name: Exported DNA SM-ID, dtype: object

In [136]:
tokeep = set(metadata['Exported DNA SM-ID']) &  set(newsamples['SM_ID'])

In [137]:
newsamples= newsamples[newsamples['SM_ID'].isin(tokeep)]
newmetadata = metadata[metadata['Exported DNA SM-ID'].isin(tokeep)]

In [138]:
newsamples = newsamples.set_index('SM_ID')

In [139]:
newmetadata = newmetadata.set_index('Exported DNA SM-ID')

In [175]:
sample_info = newsamples[['crai_or_bai_path','cram_or_bam_path']]
sample_info['reference_id'] = newsamples.index
sample_info['participant'] = newmetadata['Collaborator Participant ID']
sample_info['aggregation_product_name_validation'] = [TSCA_version]* sample_info.shape[0]
sample_info['bsp_sample_id_validation'] = newmetadata.index
sample_info['stock_sample_id_validation'] = newmetadata['Stock DNA SM-ID']
sample_info['sample_type'] = newmetadata['Sample Type']
sample_info['picard_aggregation_type_validation'] = [picard_aggregation_type_validation]* sample_info.shape[0]
sample_info['tumor_subtype'] = newmetadata['Tumor Type']
sample_info['source_subtype_validation'] = newmetadata['Original Material Type']
sample_info['processed_subtype_validation'] = newmetadata['Material Type']
sample_info['primary_disease'] = newmetadata['Primary Disease']
sample_info['media'] = newmetadata['Media on Tube']
sample_info['Collection'] = newmetadata['Collection']
sample_info['tissue_site'] = newmetadata['Tissue Site']
sample_info['source'] = [source] * sample_info.shape[0]
sample_info['sample_id'] = newmetadata['sample_id']

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [143]:
sample_info = sample_info.set_index('sample_id')

In [176]:
normals = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
tumors = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]

In [177]:
prevtumors = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type =="Tumor"]
prevnormals = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type =="Normal"]

In [190]:
# do we have new tumors/normals for our previous ones
newpairs = {'pair_id':[],'tumor_id':[],'normal_id':[]}

toreprocess = set(tumors) & set(prevnormals)
for val in toreprocess:
    for i in sample_info[sample_info['participant']==val][sample_info['sample_type']=='Tumor'].index.values:
        newpairs['pair_id'].append(i)
        newpairs['tumor_id'].append(i)
        newpairs['normal_id'].append(refsamples[refsamples['participant']==val][refsamples['sample_type']=='Normal'].index.values[0])
    
toreprocess = set(normals) & set(prevtumors)
for val in toreprocess:
    for i in refsamples[refsamples['participant']==val][refsamples['sample_type']=='Tumor'].index.values:
        newpairs['pair_id'].append(i)
        newpairs['normal_id'].append(sample_info[sample_info['participant']==val][sample_info['sample_type']=='Normal'].index.values[0])
        newpairs['tumor_id'].append(i)

paired = set(tumors) & set(normals) 
for val in paired:
    for i in sample_info[sample_info['participant']==val][sample_info['sample_type']=='Tumor'].index.values:
        newpairs['pair_id'].append(i)
        newpairs['tumor_id'].append(sample_info[sample_info['participant']==val][sample_info['sample_type']=='Tumor'].index.values[0])
        newpairs['normal_id'].append(i)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [191]:
newpairs = pd.DataFrame(newpairs)

,pair_id,tumor_id,normal_id
0,SM-J1OZD,SM-J1OZD,SM-J1OZD
1,SM-J1OZO,SM-J1OZO,SM-J1OZO
2,SM-J1OZB,SM-J1OZB,SM-J1OZB
3,SM-J1OZW,SM-J1OZW,SM-J1OZW
4,SM-J1OZZ,SM-J1OZW,SM-J1OZZ
